## Using Word2Vec and Doc2Vec to get numeric representations of YouTube video
## comments

In [1]:
!pip install -U gensim
from gensim.models import doc2vec, Doc2Vec

Requirement already up-to-date: gensim in c:\programdata\anaconda3\lib\site-packages (3.7.3)


In [2]:
from collections import OrderedDict
import pandas as pd
import re

In [4]:
videos_df = pd.read_csv('new_videos.csv')  # note that this csv data does not have comments column
videos_df.head()

,video_title,video_id,moods
0,"[FREE DL] Dave East Type Beat ""Momma Workin"" (...",kQKLl6jXipQ,CHILL
1,Logic - Indica Badu ft. Wiz Khalifa [Instrumen...,-QQUaWtMW3w,HAPPY
2,【Off Vocal】Yasuha. - I Lost Your Love feat. Ha...,1QejWtVKE8s,CHILL
3,Shiva In Exile - Earth Tone (Instrumental),4zrLieLZ_i8,CHILL
4,8 Remix - Almighty ft. Varios [INSTRUMENTAL + ...,kmLvR68_nLg,CONFIDENT-SASSY


In [27]:
comments_df = pd.read_csv('new_comments.csv', encoding="utf-8")  # note no moods column
comments_df.head()

,video_id,comments
0,2raUmsKIG6U,"['That is some cool grafiti song, sick!', 'Dam..."
1,3XAkH0w3QLc,"['the ""jake young"" at the begining scared me l..."
2,hXe0Elz7WV8,['This is good. Definitely on favorite song li...
3,x1frIzjJv5E,['I think I would sound better at the original...
4,iPqaRRdb7eQ,[]


In [7]:
# full_videos_df = pd.merge(videos_df, comments_df, on='video_id')
# use above line as example, if need to merge new_comments and new_videos csv's

In [ ]:
from nltk.tokenize import sent_tokenize

### Preprocessing and tokenizing the comments data

In [9]:
# Examine the raw comments for the first video
first_vid_comments = comments_df.comments.values[0]
first_vid_comments  # NOTE: this whole object is a string; change it to a list with strings inside

'[\'That is some cool grafiti song, sick!\', \'Damn this is dope, I miss the 90s\', \'Nice beat and cuts man, unfortunately they faded so early.\', \'Very nice beat bra\', \'Can you feel IT can you feel IT can you feeeeel itttt\', \'this is super golden age hip hop, but still so atmospheric with that string sample!! i just dropped a new vid on my channel would love to hear the opinion of a true artist like you over there! love this beat tho bro!\', \'Gj /GoodJob!\', \'Se puede sacar una super improvisacion buen beat\', "Hello. I want to use ur music for a dance project but i can\'t find you on facebook, and also not on soundcloud. How can i get your music? Thank you", "who\'s down to start a rap group", \'You bouta get the cred for some slick shit bro this beat is fucking absurd\', "It\'s ok if not", \'Can I use this brother?\', \'Hola man queria saber si podia utilizar este  beat para un tema que tengo pensado (se llama "mientras suenan las sirenas ) y es justo lo que me imagine para 

In [ ]:
# Tried various operations on first video comments
# Create a list object of first video comments by splitting on "\', " -- not successful

In [10]:
# Let's see the raw comments for the second video
second_vid_comments = comments_df.comments.values[1]
second_vid_comments

'[\'the "jake young" at the begining scared me lolol\', \'oml there are literally 4-5 comments with lyrics i cant-\', \'NNA NANANA NANAN NANANNAANNNAANANANANANANANANANANANNNNANANANANANANANANAN*YUMINOSHIi?b0AI\', \'Chi è qui per Klaus? 🇮🇹🇮🇹🇮🇹\', \'great job, can I use it for a cover? :) thank you\', \'Shut your mouth, I don’t wanna fuckin breath now\\nAll the people on my back I’m getting weighed down \\nMy future’s falling to the floor, I’m getting worried now\\nWill I make it in this game or will I fuckin drown\\n\\nI can’t fuckin sleep\\nI can’t fuckin breath \\nAll these pills are on my mind, now I’m in this deep\\nFinally passed out on the floor from all this codeine \\nShe be screaming I want more until she fuck me\\n\\nLife’s a bitter mess when you are depressed \\nAll these demons in my head I’m feeling possessed\\nI got problems with these women, yeah I’m obsessed \\nEvery day I’m working hard creating progress \\n\\nWhen I wake up and I know you’re on my fucking mind \\nI can’

In [ ]:
# Study all comments for all videos
# Based on methods from aws-sagemaker-python-notebook [not yet updated on github]
all_comments = []
all_comments_dict = {}
for comment_group in list(comments_df.comments):
    comment_group = comment_group.replace('[', '')
    comment_group = comment_group.replace(']', '')
    comment_group = comment_group.replace("'", '')
    all_comments.append(comment_group)
    # if ', ' in comment_group:
    #     comment_group = comment_group.split(', ')
    #     for item in comment_group:
    #         all_comments.append(comment_group)
    # print(comment_group, len(comment_group))

print('length of all_comments is:', len(all_comments))
print(all_comments[:2])

In [28]:
# Convert text to lower-case, do several char replacements, and create list by splitting comments string
# Create a function to do this when applied to all values in df's comments column

def normalize_text(comment_group):
    norm_comment_group = comment_group.lower()
    norm_comment_group = re.sub(r"([\.\",\(\)!\?;:])", " \\1 ", norm_comment_group)
    norm_comment_group = norm_comment_group.replace('\\n', ', ')
    norm_comment_group = norm_comment_group.replace('"', "\'")
    while "\'" in norm_comment_group:
        norm_comment_group = norm_comment_group.replace("\'", '')
    norm_comment_group = norm_comment_group.replace('[', '')
    norm_comment_group = norm_comment_group.replace(']', '')
    return norm_comment_group

# Apply normalize_text function to all comments column values

comments_df.comments = comments_df.comments.apply(lambda x: normalize_text(x))

In [29]:
comments_df.comments.values[:2]

array(['that is some cool grafiti song ,  sick !  ,  damn this is dope ,  i miss the 90s ,  nice beat and cuts man ,  unfortunately they faded so early .  ,  very nice beat bra ,  can you feel it can you feel it can you feeeeel itttt ,  this is super golden age hip hop ,  but still so atmospheric with that string sample !  !  i just dropped a new vid on my channel would love to hear the opinion of a true artist like you over there !  love this beat tho bro !  ,  gj /goodjob !  ,  se puede sacar una super improvisacion buen beat ,    hello .  i want to use ur music for a dance project but i cant find you on facebook ,  and also not on soundcloud .  how can i get your music ?  thank you   ,    whos down to start a rap group   ,  you bouta get the cred for some slick shit bro this beat is fucking absurd ,    its ok if not   ,  can i use this brother ?  ,  hola man queria saber si podia utilizar este  beat para un tema que tengo pensado  ( se llama   mientras suenan las sirenas  )  y es ju

In [30]:
# The fourth video, with id iPqaRRdb7eQ, has no comments. 
comments_df.comments.values[4]

''

In [ ]:
sent_tokenize(sent_1)  # Example of using NLTK to tokenize one video's comments as one sentence

### Trying TextBlob for NLP

In [33]:
!pip install textblob
from textblob import TextBlob

  Using cached https://files.pythonhosted.org/packages/60/f0/1d9bfcc8ee6b83472ec571406bd0dd51c0e6330ff1a51b2d29861d389e85/textblob-0.15.3-py2.py3-none-any.whl


In [184]:
from collections import Counter
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np

In [175]:
# Tags can be removed if in nltk stopwords
# To be able to iterate over stopwords, have to put them in an iterable

def tags_filter_stopwords(tags):
    stop_eng = set(stopwords.words('english'))
    stop_esp = set(stopwords.words('spanish'))
    stop_fr = set(stopwords.words('french'))
    stop_de = set(stopwords.words('german'))
    # no stopwords available for Chinese or Japanese
    stop_four = stop_eng.union(stop_esp.union(stop_fr.union(stop_de)))
    tags_list = []
    for tag in tags:
        if tag[0] not in stop_four:
            tags_list.append(tag)
    return tags_list


# Also want to know which parts of speech TextBlob codes are most common among the tags
def top_three_tag_codes(tags):
    """Receives list of tags; returns 3 most common TextBlob parts-of-speech codes"""
    # Recall that each TextBlob tag is a tuple of a word and a TextBlob part-of-speech code
    speech_parts_list = []
    for tag in tags:
        speech_parts_list.append(tag[1])
    c = Counter(speech_parts_list)
    print(c.most_common(3), 'len(c.most_common(3)) is:', len(c.most_common(3)))
    # note that there may only be one most common tag
    return c.most_common(3)  # returns a list of 2-item tuple[s] with code and count for code in tags

In [185]:
# Add two new columns to comments_df: one each for TextBlob polarity mean and subjectivity mean
# Want to add other new columns also based on tags:
# one column for each of the top 3, and one column for the count in comments for each of the top 3 

tags_columns = ['polarity_mean', 'subjectivity_mean', 'tags', 'top_tag', 'top_tag_count',
                'second_tag', 'second_tag_count', 'third_tag', 'third_tag_count']
analysis_df = pd.concat([comments_df, pd.DataFrame(columns=tags_columns)], axis=1)

analysis_df = pd.merge(analysis_df, videos_df, on='video_id')
cols = analysis_df.columns.to_list()
cols = [cols[0]] + [cols[11]] + [cols[1]] + cols[2:11] + [cols[-1]]
analysis_df = analysis_df[cols]

In [144]:
analysis_df.head()

,video_id,video_title,comments,polarity_mean,subjectivity_mean,tags,top_tag,top_tag_count,second_tag,second_tag_count,third_tag,third_tag_count,moods
0,2raUmsKIG6U,"[SOLD]"" Graffiti "" - 90s Old School Storytelli...","that is some cool grafiti song , sick ! , d...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHILL
1,3XAkH0w3QLc,Youth by Daughter | Instrumental (w/ Sheet Music),the jake young at the begining scared me l...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHILL
2,hXe0Elz7WV8,THE CREATOR LIED TO US (Fansong Instrumental) ...,this is good . definitely on favorite song li...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFIDENT-SASSY
3,x1frIzjJv5E,Kendrick Lamar - Blood. (RoadsArt Instrumental...,i think i would sound better at the original s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHILL
4,iPqaRRdb7eQ,Nco Qub Hluas Nraug Instrumental + Lyrics - Gi...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHILL


In [186]:
# Drop rows where there are no comments
analysis_df = analysis_df[analysis_df.comments != '']
analysis_df.shape

(86, 13)

In [187]:
# Add TextBlob mean polarity, mean subjectivity, and tags features for all comments for each video
# Write functions to do each of the following:
# 1) run TextBlob on comments; 2) populate polarity_mean, subjectivity_mean, and tags values

def get_blob(comments):
    """Use with one video's comments at a time."""
    comments = normalize_text(comments)
    blob = TextBlob(comments)
    tags = tags_filter_stopwords(blob.tags)
    top_tags = top_three_tag_codes(tags)  # this is a list of 2-item tuple[s]
    top_tag = top_tags[0][0]
    top_tag_count = top_tags[0][1]
    if len(top_tags) < 2:
        second_tag = np.nan
        second_tag_count = 0
        third_tag = np.nan
        third_tag_count = 0
    elif len(top_tags) == 2:
        second_tag = top_tags[1][0]
        second_tag_count = top_tags[1][1]
        third_tag = np.nan
        third_tag_count = 0
    elif len(top_tags) > 2:
        # 3-item list of tuples with each top code and its count
        second_tag = top_tags[1][0]
        second_tag_count = top_tags[1][1]
        third_tag = top_tags[2][0]
        third_tag_count = top_tags[2][1]
    polarity_list = []
    subjectivity_list = []
    for sentence in blob.sentences:
        polarity = sentence.sentiment.polarity
        subjectivity = sentence.sentiment.subjectivity
        if polarity !=0 :
            polarity_list.append(polarity)
        if subjectivity !=0 :
            subjectivity_list.append(subjectivity)
    if len(polarity_list) == 0:
        polarity_list.append(0)
    if len(subjectivity_list) == 0:
        subjectivity_list.append(0)
    # print(top_tag, second_tag)
    return (mean(polarity_list), mean(subjectivity_list), tags, top_tag, top_tag_count,
            second_tag, second_tag_count, third_tag, third_tag_count)

def populate_blob_columns(comms):
    """Use to fill the polarity_mean, subjectivity_mean, and tags columns."""
    (polarity_mean, subjectivity_mean, tags, top_tag, top_tag_count,
            second_tag, second_tag_count, third_tag, third_tag_count) = get_blob(comms)
    analysis_df.loc[(analysis_df.comments == comms), 'polarity_mean'] = polarity_mean
    analysis_df.loc[(analysis_df.comments == comms), 'subjectivity_mean'] = subjectivity_mean
    analysis_df.loc[(analysis_df.comments == comms), 'tags'] = [tags]
    analysis_df.loc[(analysis_df.comments == comms), 'top_tag'] = top_tag
    analysis_df.loc[(analysis_df.comments == comms), 'top_tag_count'] = top_tag_count
    analysis_df.loc[(analysis_df.comments == comms), 'second_tag'] = second_tag
    analysis_df.loc[(analysis_df.comments == comms), 'second_tag_count'] = second_tag_count
    analysis_df.loc[(analysis_df.comments == comms), 'third_tag'] = third_tag
    analysis_df.loc[(analysis_df.comments == comms), 'third_tag_count'] = third_tag_count
    
    
for comments in analysis_df.comments.values:
    populate_blob_columns(comments)

[('NN', 67), ('JJ', 25), ('VB', 14)] len(c.most_common(3)) is: 3
[('NN', 85), ('JJ', 32), ('VBP', 31)] len(c.most_common(3)) is: 3
[('NN', 22), ('JJ', 10), ('NNS', 4)] len(c.most_common(3)) is: 3
[('NN', 19), ('JJ', 7), ('VB', 3)] len(c.most_common(3)) is: 3
[('NN', 8)] len(c.most_common(3)) is: 1
[('NN', 5), ('JJ', 4), ('IN', 1)] len(c.most_common(3)) is: 3
[('NN', 5), ('VBD', 1), ('VBP', 1)] len(c.most_common(3)) is: 3
[('NN', 24), ('JJ', 7), ('VB', 5)] len(c.most_common(3)) is: 3
[('NN', 224), ('JJ', 39), ('VB', 37)] len(c.most_common(3)) is: 3
[('NN', 148), ('VB', 72), ('JJ', 51)] len(c.most_common(3)) is: 3
[('NN', 69), ('VB', 23), ('JJ', 23)] len(c.most_common(3)) is: 3
[('NN', 131), ('JJ', 66), ('VBP', 65)] len(c.most_common(3)) is: 3
[('NN', 273), ('JJ', 73), ('NNS', 53)] len(c.most_common(3)) is: 3
[('NN', 8), ('JJ', 3), ('FW', 3)] len(c.most_common(3)) is: 3
[('NN', 5), ('VBP', 3)] len(c.most_common(3)) is: 2
[('NN', 172), ('NNS', 60), ('JJ', 51)] len(c.most_common(3)) is: 3


In [188]:
# Check how df looks
print(analysis_df.shape)
print(analysis_df.isna().sum().sum())
analysis_df.head()

(86, 13)
9


,video_id,video_title,comments,polarity_mean,subjectivity_mean,tags,top_tag,top_tag_count,second_tag,second_tag_count,third_tag,third_tag_count,moods
0,2raUmsKIG6U,"[SOLD]"" Graffiti "" - 90s Old School Storytelli...","that is some cool grafiti song , sick ! , d...",0.252059,0.652953,"[(cool, JJ), (grafiti, NN), (song, NN), (sick,...",NN,67,JJ,25,VB,14,CHILL
1,3XAkH0w3QLc,Youth by Daughter | Instrumental (w/ Sheet Music),the jake young at the begining scared me l...,0.45385,0.707216,"[(jake, NN), (young, JJ), (begining, NN), (sca...",NN,85,JJ,32,VBP,31,CHILL
2,hXe0Elz7WV8,THE CREATOR LIED TO US (Fansong Instrumental) ...,this is good . definitely on favorite song li...,0.625,0.733333,"[(good, JJ), (definitely, RB), (favorite, NN),...",NN,22,JJ,10,NNS,4,CONFIDENT-SASSY
3,x1frIzjJv5E,Kendrick Lamar - Blood. (RoadsArt Instrumental...,i think i would sound better at the original s...,0.2875,0.811389,"[(think, VBP), (would, MD), (sound, VB), (bett...",NN,19,JJ,7,VB,3,CHILL
5,qL034NgzGxY,Cim Ntsoov Cia Instrumental + Lyrics,zoo heev li kho siab heev zoo mloog,0,0,"[(zoo, NN), (heev, NN), (li, NN), (kho, NN), (...",NN,8,NaN,0,NaN,0,SAD


#### Apply ML

In [189]:
# First normalize the data by multiplying polarity_mean and subjectivity_mean each by 100
analysis_df.polarity_mean *= 100
analysis_df.subjectivity_mean *= 100

# Verify that the values have been scaled by 100
analysis_df.polarity_mean.values[:2]

array([25.205943362193363, 45.38504088504089], dtype=object)

In [196]:
# Next, define features for X, and target for y
# And split data into training and test sets

X = analysis_df[['polarity_mean', 'subjectivity_mean', 'top_tag_count',
                 'second_tag_count', 'third_tag_count']]
y = analysis_df.moods

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [197]:
# Fit a model - this is a multiclass classification, supervised machine learning problem
# because there are 6 moods in the df

# Try a simple model first - KNN classifier
from sklearn.neighbors import KNeighborsClassifier

In [198]:
knn = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)

In [199]:
# Check accuracy of the model, using X_test and y_test
accuracy = knn.score(X_test, y_test)
print('accuracy is:', accuracy)

accuracy is: 0.09090909090909091


In [202]:
# Now will try a more complex model - Random Forest classification
from sklearn.ensemble import RandomForestClassifier

In [203]:
RFC = RandomForestClassifier(n_estimators=25, max_depth=2, random_state=7)

In [204]:
RFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=None,
            oob_score=False, random_state=7, verbose=0, warm_start=False)

In [205]:
# See the respective importance for each of the features in X
print(RFC.feature_importances_)

[0.31251438 0.09514156 0.24587961 0.19035721 0.15610723]


In [206]:
# Check accuracy of the model
accuracy = RFC.score(X_test, y_test)
print('accuracy is:', accuracy)

accuracy is: 0.13636363636363635


In [207]:
# Send a csv
analysis_df.to_csv('model_training.csv')

#### Sandbox for working with new comments, TextBlob, other

In [136]:
# Let's work with the first video's comments, to start

blob = TextBlob(comments_df.comments.values[0])

In [137]:
# See if tags can be gotten for the first video's comments
print(len(blob.tags))
blob.tags[:3]

246


[('that', 'DT'), ('is', 'VBZ'), ('some', 'DT')]

In [138]:
tags_filter_stopwords(blob.tags)

[('cool', 'JJ'),
 ('grafiti', 'NN'),
 ('song', 'NN'),
 ('sick', 'NN'),
 ('damn', 'NN'),
 ('dope', 'NN'),
 ('miss', 'VBP'),
 ('90s', 'CD'),
 ('nice', 'JJ'),
 ('beat', 'NN'),
 ('cuts', 'NNS'),
 ('unfortunately', 'RB'),
 ('faded', 'VBD'),
 ('early', 'JJ'),
 ('nice', 'JJ'),
 ('beat', 'NN'),
 ('bra', 'NN'),
 ('feel', 'VB'),
 ('feel', 'VB'),
 ('feeeeel', 'VB'),
 ('itttt', 'RB'),
 ('super', 'JJ'),
 ('golden', 'JJ'),
 ('age', 'NN'),
 ('hip', 'NN'),
 ('hop', 'NN'),
 ('still', 'RB'),
 ('atmospheric', 'JJ'),
 ('string', 'VBG'),
 ('sample', 'NN'),
 ('dropped', 'VBD'),
 ('new', 'JJ'),
 ('vid', 'NN'),
 ('channel', 'NN'),
 ('would', 'MD'),
 ('love', 'VB'),
 ('hear', 'VB'),
 ('opinion', 'NN'),
 ('true', 'JJ'),
 ('artist', 'NN'),
 ('like', 'IN'),
 ('love', 'VB'),
 ('beat', 'NN'),
 ('tho', 'JJ'),
 ('bro', 'NN'),
 ('gj', 'JJ'),
 ('/goodjob', 'NN'),
 ('puede', 'NN'),
 ('sacar', 'NN'),
 ('super', 'JJ'),
 ('improvisacion', 'NN'),
 ('buen', 'NN'),
 ('beat', 'NN'),
 ('hello', 'NN'),
 ('want', 'VBP'),
 ('use',

In [150]:
# Verify correct functioning of top_three_tag_codes
top_three_tag_codes(blob.tags)

[('NN', 81), ('JJ', 32), ('DT', 18)]

In [ ]:
# Exploring sentiment delivered by TextBlob, on 15 May 2019

from statistics import mean

polarity_list = []
subjectivity_list = []
for sentence in blob.sentences:
    print('polarity and subjectivity for this sentence are:', sentence.sentiment)
    polarity = sentence.sentiment.polarity
    subjectivity = sentence.sentiment.subjectivity
    if polarity !=0 :
        polarity_list.append(polarity)
    if subjectivity !=0 :
        subjectivity_list.append(subjectivity)
print('polarity_list is:', polarity_list, '\n', 'subjectivity_list is:', subjectivity_list)

In [97]:
# Looking at a new comments csv from 16 May 2019

comments1_df = pd.read_csv('new_comments1.csv')
comments1_df.head()
len(comments1_df.loc[comments1_df.comments == '[]'])

72

In [124]:
get_blob(comments_df.comments.values[0])

(0.2520594336219336,
 0.6529527417027416,
 [('that', 'DT'),
  ('is', 'VBZ'),
  ('some', 'DT'),
  ('cool', 'JJ'),
  ('grafiti', 'NN'),
  ('song', 'NN'),
  ('sick', 'NN'),
  ('damn', 'NN'),
  ('this', 'DT'),
  ('is', 'VBZ'),
  ('dope', 'NN'),
  ('i', 'JJ'),
  ('miss', 'VBP'),
  ('the', 'DT'),
  ('90s', 'CD'),
  ('nice', 'JJ'),
  ('beat', 'NN'),
  ('and', 'CC'),
  ('cuts', 'NNS'),
  ('man', 'NN'),
  ('unfortunately', 'RB'),
  ('they', 'PRP'),
  ('faded', 'VBD'),
  ('so', 'RB'),
  ('early', 'JJ'),
  ('very', 'RB'),
  ('nice', 'JJ'),
  ('beat', 'NN'),
  ('bra', 'NN'),
  ('can', 'MD'),
  ('you', 'PRP'),
  ('feel', 'VB'),
  ('it', 'PRP'),
  ('can', 'MD'),
  ('you', 'PRP'),
  ('feel', 'VB'),
  ('it', 'PRP'),
  ('can', 'MD'),
  ('you', 'PRP'),
  ('feeeeel', 'VB'),
  ('itttt', 'RB'),
  ('this', 'DT'),
  ('is', 'VBZ'),
  ('super', 'JJ'),
  ('golden', 'JJ'),
  ('age', 'NN'),
  ('hip', 'NN'),
  ('hop', 'NN'),
  ('but', 'CC'),
  ('still', 'RB'),
  ('so', 'RB'),
  ('atmospheric', 'JJ'),
  ('with', 'I

In [159]:
type(comments_df.comments[4])
# get_blob(comments_df.comments[4])

str

In [178]:
get_blob(comments_df.comments[5])

[('NN', 8)] len(c.most_common(3)) is: 1


UnboundLocalError: local variable 'second_tag' referenced before assignment

#### End Sandbox

### Creating a Doc2Vec NLP model

In [ ]:
# make a Doc2Vec TaggedDocument for each video's comments;
# append each of those Doc2Vec TaggedDocuments to a "sentences" list;
# create: model = models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1);
# call: model.build_vocab(sentences);
# train model: for epoch in range(int):... model.train, model.alpha, model.min_alpha

In [31]:
sentences = []
sentences_ordered_dict = OrderedDict()
sent_count = 0
raw_comments = list(comments_df.comments.values)  # this is all of the comments for all videos
for comment_group in raw_comments:
    sent_key = 'sentence' + str(sent_count)
    sentences_ordered_dict[sent_key] = doc2vec.TaggedDocument(
        words=comment_group, tags=["SENT_" + str(sent_count)])
    sentences.append(sentences_ordered_dict[sent_key])
    sent_count += 1
print('TaggedDocument count in sentences is:', len(sentences), '\n')
print(sentences[0])

TaggedDocument count in sentences is: 93 

TaggedDocument(that is some cool grafiti song ,  sick !  ,  damn this is dope ,  i miss the 90s ,  nice beat and cuts man ,  unfortunately they faded so early .  ,  very nice beat bra ,  can you feel it can you feel it can you feeeeel itttt ,  this is super golden age hip hop ,  but still so atmospheric with that string sample !  !  i just dropped a new vid on my channel would love to hear the opinion of a true artist like you over there !  love this beat tho bro !  ,  gj /goodjob !  ,  se puede sacar una super improvisacion buen beat ,    hello .  i want to use ur music for a dance project but i cant find you on facebook ,  and also not on soundcloud .  how can i get your music ?  thank you   ,    whos down to start a rap group   ,  you bouta get the cred for some slick shit bro this beat is fucking absurd ,    its ok if not   ,  can i use this brother ?  ,  hola man queria saber si podia utilizar este  beat para un tema que tengo pensado  ( 

In [ ]:
# Create doc2vec model and build its vocabulary
model = Doc2Vec(alpha=.025, min_alpha=.025, min_count=2)
model.build_vocab(sentences)

In [ ]:
# Get more info about the model
print(model)

In [ ]:
# Find out which objects related to vocabulary are available in the Doc2Vec namespace
[x for x in dir(model) if 'vocab' in x]

In [ ]:
# Look closer at 'vocabulary'
help(model.vocabulary)

In [ ]:
# infer new tokens on new comments documents